In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy
import math
import pandas_ta as ta
from pathlib import Path
import yfinance as yf
from datetime import datetime, timedelta

In [167]:
nas_path = Path('./nasdaq_2019_to_present.csv')
nas_df = pd.read_csv(nas_path)
nas_df['date'] = pd.to_datetime(nas_df['date'])
nas_df = nas_df.set_index('date')
nas_df.columns = ['close', 'open', 'high', 'low']
nas_df

,close,open,high,low
date,,,,
2024-01-21,17462.75,17462.75,17462.75,17462.75
2024-01-20,17462.75,17462.75,17462.75,17462.75
2024-01-19,17438.50,17116.00,17471.25,17107.25
2024-01-18,17110.00,16854.50,17125.75,16834.25
2024-01-17,16869.75,16978.00,16982.00,16689.25
...,...,...,...,...
2020-07-07,10532.25,10600.00,10694.50,10505.75
2020-07-06,10598.50,10347.25,10614.50,10341.50
2020-07-03,10328.88,10348.00,10389.25,10310.20


In [168]:
end_date = datetime.now() #adds current date

# Subtract 1825 days (5 years) from the end_date to get the start_date.
start_date = end_date - timedelta(days = 1825)

# Convert the start_date to a string in the format 'YYYY-MM-DD'.
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

In [169]:
security_df = yf.Ticker('NQ=F')

security_df.info # Accessing the information of the NASDAQ Futures. 

hist = security_df.history(start = start_date_str, end =  end_date_str, interval = '1D') # start = '2023-11-23', end = '2024-01-21'
                                                                 #Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
nasdaq_df = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']])
df_cumulative_delta = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']])
nasdaq_df


,Volume,Close,High,Low
Date,,,,
2019-01-22 00:00:00-05:00,651258,6653.00,6794.50,6611.50
2019-01-23 00:00:00-05:00,515044,6665.25,6724.25,6592.25
2019-01-24 00:00:00-05:00,449017,6675.00,6718.50,6653.00
2019-01-25 00:00:00-05:00,446822,6791.25,6810.50,6676.75
2019-01-28 00:00:00-05:00,490045,6701.75,6791.00,6648.25
...,...,...,...,...
2024-01-15 00:00:00-05:00,593445,16970.00,17024.25,16936.25
2024-01-16 00:00:00-05:00,816021,16966.50,17033.75,16811.75
2024-01-17 00:00:00-05:00,722784,16869.75,16982.00,16689.25


In [170]:
def find_levels(
        price: np.array,       # Array of log closing prices.
        atr: float,            # Average True Range (ATR), a measure of market volatility.
        first_w: float = 0.1,  # Initial weight for the weighted kernel density estimation.
        atr_mult: float = 3.0,  # Multiplier for ATR to adjust bandwidth in kernel density estimation.
        prom_thresh: float = 0.1  # Threshold for prominence in peak finding, relative to the maximum of the probability density function.
):

    # Setup for weights in the kernel density estimation.
    last_w = 1.0  # Final weight.
    w_step = (last_w - first_w) / len(price)  # Step size for weights based on the number of price points.
    weights = first_w + np.arange(len(price)) * w_step  # Linearly increasing weights array.
    weights[weights < 0] = 0.0  # Ensuring no negative weights.

    # Gaussian Kernel Density Estimation on the price data with weights and adjusted bandwidth.
    # Kernel Density Estimation is a non-parametric way to estimate the probability density function (PDF) of a random variable. 
    # In simple terms, it's a technique used to visualize the distribution of data points. 
    # The Gaussian KDE uses the Gaussian (normal) distribution as the kernel. 
    # This means it approximates the distribution of your data points by summing up Gaussian distributions (bell curves) centered at each data point.
    kernel = scipy.stats.gaussian_kde(price, bw_method=atr*atr_mult, weights=weights)

    # Construct market profile by evaluating the kernel density across a range of prices.
    min_v = np.min(price)  # Minimum price in the range.
    max_v = np.max(price)  # Maximum price in the range.
    step = (max_v - min_v) / 200  # Step size for the price range.
    price_range = np.arange(min_v, max_v, step)  # Price range for evaluation.
    pdf = kernel(price_range)  # Market profile as a probability density function.

    # Find significant peaks in the market profile.
    pdf_max = np.max(pdf)  # Maximum value in the probability density function.
    prom_min = pdf_max * prom_thresh  # Minimum prominence for a peak to be considered significant.

    # Finding peaks in the market profile with the defined minimum prominence.
    peaks, props = scipy.signal.find_peaks(pdf, prominence=prom_min)

    # Converting the log price back to the actual price at each peak to identify levels.
    levels = [np.exp(price_range[peak]) for peak in peaks]

    return levels, peaks, props, price_range, pdf, weights  # Return identified levels and related data.


def support_resistance_levels(
        data,                 # DataFrame containing the financial data.
        lookback: int,        # The number of periods to look back for calculating ATR and levels.
        first_w: float = 0.03,  # Initial weight for kernel density estimation in find_levels.
        atr_mult: float = 3.5,  # Multiplier for ATR to adjust bandwidth in kernel density estimation.
        prom_thresh: float = 0.20  # Threshold for prominence in peak finding.
):
    # Check if necessary columns are present in the DataFrame.
    if 'high' not in data.columns or 'low' not in data.columns or 'close' not in data.columns:
        raise KeyError("Columns 'high', 'low', and 'close' must be present in the DataFrame.")

    # Calculate the logarithmic Average True Range (ATR) for the given lookback period.
    atr = ta.atr(np.log(data['high']), np.log(data['low']), np.log(data['close']), lookback)

    # Initialize a list to store the levels for each data point.
    all_levels = [None] * len(data)

    # Iterate over the data, starting from the lookback period to the end of the data.
    for i in range(lookback, len(data)):
        i_start = i - lookback  # Determine the starting index for the lookback period.
        
        # Extract logarithmic closing prices for the lookback period.
        vals = np.log(data.iloc[i_start + 1: i + 1]['close'].to_numpy())

        # Calculate the support and resistance levels using the find_levels function.
        levels, peaks, props, price_range, pdf, weights = find_levels(
            vals, atr.iloc[i], first_w, atr_mult, prom_thresh
        )

        # Store the calculated levels for the current index.
        all_levels[i] = levels

    return all_levels  

def sr_penetration_signal(data, levels: list):
    # Initialize a signal array with zeros, same length as the data.
    signal = np.zeros(len(data))

    # Variable to store the current signal state.
    curr_sig = 0.0

    # Convert the 'close' column of the data to a numpy array for efficiency.
    close_arr = data['close'].to_numpy()

    # Iterate through each data point, starting from the second one (index 1).
    for i in range(1, len(data)):
        # If there are no levels identified for the current data point, skip to the next.
        if levels[i] is None:
            continue
        # Store the previous and current closing prices.
        last_c = close_arr[i - 1]
        curr_c = close_arr[i]
        # Check for penetration of each level identified for the current data point.
        for level in levels[i]:
            # If the current close is above the level and the last close was at or below it,
            # it indicates a bullish signal (cross above the support/resistance level).
            if curr_c > level and last_c <= level:
                curr_sig = 1.0
            # If the current close is below the level and the last close was at or above it,
            # it indicates a bearish signal (cross below the support/resistance level).
            elif curr_c < level and last_c >= level:
                curr_sig = -1.0

        # Update the signal array for the current index with the current signal state.
        signal[i] = curr_sig
    # Return the array containing the signals.
    return signal




def get_trades_from_signal(data, signal: np.array):
    # Initialize lists to store long and short trades.
    long_trades = []
    short_trades = []

    # Convert the 'close' column of the data to a numpy array for efficient processing.
    close_arr = data['close'].to_numpy()

    # Variable to store the last signal processed.
    last_sig = 0.0

    # Variable to keep track of the current open trade.
    open_trade = None

    # Get the index of the DataFrame, which represents time.
    idx = data.index

    # Iterate through each data point.
    for i in range(len(data)):
        # Check for a long entry signal and that it's not a continuation of the previous signal.
        if signal[i] == 1.0 and last_sig != 1.0:  # Long entry
            # Close any open short trade.
            if open_trade is not None:
                open_trade[2] = idx[i]  # Exit time.
                open_trade[3] = close_arr[i]  # Exit price.
                short_trades.append(open_trade)

            # Open a new long trade.
            open_trade = [idx[i], close_arr[i], -1, np.nan]  # Initialize with entry time and price.

        # Check for a short entry signal and that it's not a continuation of the previous signal.
        if signal[i] == -1.0 and last_sig != -1.0:  # Short entry
            # Close any open long trade.
            if open_trade is not None:
                open_trade[2] = idx[i]  # Exit time.
                open_trade[3] = close_arr[i]  # Exit price.
                long_trades.append(open_trade)

            # Open a new short trade.
            open_trade = [idx[i], close_arr[i], -1, np.nan]  # Initialize with entry time and price.

        # Update the last signal variable.
        last_sig = signal[i]

    # Convert the lists of trades to DataFrames for easier manipulation and analysis.
    long_trades = pd.DataFrame(long_trades, columns=['entry_time', 'entry_price', 'exit_time', 'exit_price'])
    short_trades = pd.DataFrame(short_trades, columns=['entry_time', 'entry_price', 'exit_time', 'exit_price'])

    # Calculate the percentage gain/loss for each trade.
    long_trades['percent'] = (long_trades['exit_price'] - long_trades['entry_price']) / long_trades['entry_price']
    short_trades['percent'] = -1 * (short_trades['exit_price'] - short_trades['entry_price']) / short_trades['entry_price']

    # Set the entry time as the index for both DataFrames.
    long_trades = long_trades.set_index('entry_time')
    short_trades = short_trades.set_index('entry_time')

    return long_trades, short_trades

In [171]:
if __name__ == '__main__': # This condition checks if the script is being run as the main program and not imported as a module.

    # Assigning 'nas_df' to 'data'. 'nas_df' should be a DataFrame containing financial data.
    data = nas_df

    # Set the plotting style to 'dark_background' for better visualization.
    plt.style.use('dark_background')

    # Calculate support and resistance levels using the 'support_resistance_levels' function.
    # It uses a 365-day lookback period, with specific parameters for weighted kernel density estimation.
    levels = support_resistance_levels(data, 365, first_w=1.0, atr_mult=3.0)

    # Generate trading signals based on the penetration of support and resistance levels.
    data['sr_signal'] = sr_penetration_signal(data, levels)

    # Calculate the logarithmic return of the 'close' price, shifting the result by -1 to align with the next period.
    data['log_ret'] = np.log(data['close']).diff().shift(-1)

    # Calculate the return from the strategy by multiplying the signal by the logarithmic return.
    data['sr_return'] = data['sr_signal'] * data['log_ret']
    
    # Generate lists of long and short trades based on the trading signals.
    long_trades, short_trades = get_trades_from_signal(data, data['sr_signal'].to_numpy())    

In [173]:
#Calcualting HLC3 value
nasdaq_df['HLC3'] = (nasdaq_df['High'] + nasdaq_df['Low'] + nasdaq_df['Close']) / 3
# Calculating VWAP (using HLC3)
nasdaq_df['VWAP'] = (nasdaq_df['HLC3'] * nasdaq_df['Volume']).cumsum() / nasdaq_df['Volume'].cumsum()
#Calculating Standard Deviation
nasdaq_df['PriceDeviation'] = (nasdaq_df['HLC3'] - nasdaq_df['VWAP']).rolling(window = 78).std() ##Window 78 set based on 5 min trading data (78 5 min in 1 day)##
# Session based (How it was on Pinescript) appears to be based on daily
# Calculate bands [UpperBand_1, LowerBand_1, UpperBand_2, LowerBand_2, UpperBand_3, LowerBand_3]
for i in [1, 2, 3]:
    nasdaq_df[f'UpperBand_{i}'] = nasdaq_df['VWAP'] + (i * nasdaq_df['PriceDeviation'])
    nasdaq_df[f'LowerBand_{i}'] = nasdaq_df['VWAP'] - (i * nasdaq_df['PriceDeviation'])
nasdaq_df = nasdaq_df.dropna()
df_hlc3_vwap = nasdaq_df
df_hlc3_vwap

,Volume,Close,High,Low,HLC3,VWAP,PriceDeviation,UpperBand_1,LowerBand_1,UpperBand_2,LowerBand_2,UpperBand_3,LowerBand_3
Date,,,,,,,,,,,,,
2019-05-13 00:00:00-04:00,773473,7314.75,7551.50,7305.00,7390.416667,7312.836158,176.944687,7489.780845,7135.891472,7666.725532,6958.946785,7843.670219,6782.002098
2019-05-14 00:00:00-04:00,547769,7421.00,7459.50,7290.00,7390.166667,7314.040518,175.458428,7489.498947,7138.582090,7664.957375,6963.123661,7840.415804,6787.665233
2019-05-15 00:00:00-04:00,535915,7529.25,7538.50,7353.75,7473.833333,7316.438764,172.470163,7488.908927,7143.968601,7661.379091,6971.498438,7833.849254,6799.028275
2019-05-16 00:00:00-04:00,525588,7600.25,7641.00,7473.50,7571.583333,7320.139827,169.120542,7489.260369,7151.019285,7658.380911,6981.898743,7827.501453,6812.778201
2019-05-17 00:00:00-04:00,626777,7511.50,7620.00,7506.50,7546.000000,7323.980425,167.096717,7491.077142,7156.883708,7658.173859,6989.786991,7825.270577,6822.690274
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-15 00:00:00-05:00,593445,16970.00,17024.25,16936.25,16976.833333,12182.785209,767.416123,12950.201332,11415.369087,13717.617455,10647.952964,14485.033577,9880.536842
2024-01-16 00:00:00-05:00,816021,16966.50,17033.75,16811.75,16937.333333,12188.392927,769.918357,12958.311284,11418.474570,13728.229641,10648.556213,14498.147998,9878.637856
2024-01-17 00:00:00-05:00,722784,16869.75,16982.00,16689.25,16847.000000,12193.254610,769.109108,12962.363718,11424.145502,13731.472827,10655.036394,14500.581935,9885.927286


In [ ]:
# Calculate the total points gained or lost in each long trade: (exit price - entry price).
long_trades['total_points'] = long_trades['exit_price'] - long_trades['entry_price']
# Calculate the profit and loss (PnL). Here, '20' is the point value per contract for the futures.
# Therefore, each point movement in the price results in a $20 gain or loss per contract.
long_trades['pnl_1_contract'] = long_trades['total_points'] * 20
long_trades

,entry_price,exit_time,exit_price,percent,total_points,pnl_1_contract
entry_time,,,,,,
2022-09-21,11710.00,2022-08-09,13031.50,0.112852,1321.50,26430.0
2022-08-05,13228.75,2022-08-02,12924.50,-0.022999,-304.25,-6085.0
2022-07-08,12152.00,2022-07-06,11880.25,-0.022363,-271.75,-5435.0
2022-06-29,11691.00,2022-06-22,11565.75,-0.010713,-125.25,-2505.0
2022-06-10,11840.00,2022-03-28,14985.25,0.265646,3145.25,62905.0
2021-11-08,16327.75,2021-11-03,16129.75,-0.012127,-198.00,-3960.0
2021-10-01,14761.75,2021-09-30,14682.50,-0.005369,-79.25,-1585.0
2021-09-29,14739.75,2021-08-31,15582.50,0.057175,842.75,16855.0
2021-08-30,15597.50,2021-08-28,15428.25,-0.010851,-169.25,-3385.0


In [ ]:
#calculate net profit of all long positions
long_pnl = long_trades['pnl_1_contract'].sum()
long_pnl

60590.0

In [ ]:
# Calculate the total points gained or lost in each long trade.
short_trades['total_points'] = short_trades['entry_price'] - short_trades['exit_price']
# Calculate the profit and loss (PnL). Here, '20' is the point value per contract for the futures.
# Therefore, each point movement in the price results in a $20 gain or loss per contract.
short_trades['pnl_1_contract'] = short_trades['total_points'] * 20
short_trades

,entry_price,exit_time,exit_price,percent,total_points,pnl_1_contract
entry_time,,,,,,
2022-08-09,13031.50,2022-08-05,13228.75,-0.015136,-197.25,-3945.0
2022-08-02,12924.50,2022-07-08,12152.00,0.059770,772.50,15450.0
2022-07-06,11880.25,2022-06-29,11691.00,0.015930,189.25,3785.0
2022-06-22,11565.75,2022-06-10,11840.00,-0.023712,-274.25,-5485.0
2022-03-28,14985.25,2021-11-08,16327.75,-0.089588,-1342.50,-26850.0
2021-11-03,16129.75,2021-10-01,14761.75,0.084812,1368.00,27360.0
2021-09-30,14682.50,2021-09-29,14739.75,-0.003899,-57.25,-1145.0
2021-08-31,15582.50,2021-08-30,15597.50,-0.000963,-15.00,-300.0
2021-08-28,15428.25,2021-07-29,15037.75,0.025311,390.50,7810.0


In [ ]:
#calculate net profit of all short positions
short_pnl = short_trades['pnl_1_contract'].sum()
short_pnl

72955.0